In [1]:
import geopandas as gpd
from shapely.geometry import Polygon
import pandas as pd
import pharmada.data as data
import pharmada.regkey as rk
import math
import re
import csv

regkey = '09663'

In [2]:
data = data.Data(regkey)

In [3]:
file = open('./data/Zensus_Bevoelkerung_100m-Gitter.csv', 'r', newline='')
reader = csv.reader(file, delimiter=';')

In [4]:
area_geom = data.AreaGeometry.geometry
area_geom.to_crs(epsg=3035, inplace=True)
bounds = area_geom.total_bounds

e_sw = int(bounds[0]) // 10**2 * 10**2 + 50
n_sw = int(bounds[1]) // 10**2 * 10**2 + 50
e_ne = int(bounds[2]) // 10**2 * 10**2 - 50
n_ne = int(bounds[3]) // 10**2 * 10**2 - 50

def in_range(x, y):
    return n_sw <= y <= n_ne and e_sw <= x <= e_ne

In [5]:
gridcells = []

next(reader) # skip header

for row in reader:
    if in_range(int(row[1]), int(row[2])):
        gridcells.append(row)

In [6]:
gridcells = pd.DataFrame(gridcells)
gridcells.rename(columns={0: 'id', 1: 'x_mp', 2: 'y_mp', 3: 'population'}, inplace=True)
gridcells['x_mp'] = gridcells['x_mp'].astype(int)
gridcells['y_mp'] = gridcells['y_mp'].astype(int)
gridcells['population'] = gridcells['population'].astype(int)

In [7]:
gridcells['geometry'] = gridcells.apply(lambda row: Polygon([(row['x_mp']-50, row['y_mp']-50),
                                     (row['x_mp']+50, row['y_mp']-50),
                                     (row['x_mp']+50, row['y_mp']+50),
                                     (row['x_mp']-50, row['y_mp']+50)]), axis=1)
gridcells = gpd.GeoDataFrame(gridcells, geometry='geometry')
gridcells.set_crs(epsg=3035, inplace=True)

to_drop = []
for index, row in gridcells.iterrows():
    if not row['population'] > 0:
        to_drop.append(index)
    elif not area_geom.contains(row.geometry)[0]:
        to_drop.append(index)

gridcells.drop(to_drop, inplace=True)

In [10]:
current_population = rk.get_regkey_list().at[data.RegKey.regkey, 'population']
current_population

126933